In [1]:
import time

import torch
import torch.nn as nn
import torch.utils.data
from eval_cifar10 import Eval
from models import resnet50

In [2]:
evaluator = Eval(batch_size=32)

model = resnet50(pretrained=True)
model.eval()

# 通常のモデル
start = time.time()
print(evaluator.eval(model, torch.device("cpu")))
# CIFAR-10 精度 93.58 %
print(time.time() - start)
# Intel Core i7-12700 で 27.6 秒

Files already downloaded and verified
0.9358
26.5885066986084


In [3]:
def low_rank_recursively(module, rank=32):
    for name, child in module.named_children():
        if isinstance(child, nn.Conv2d):
            kernel_size = child.kernel_size
            in_channels = child.in_channels
            out_channels = child.out_channels
            has_bias = child.bias is not None
            W = child.weight.permute(0, 2, 1, 3).reshape(out_channels * kernel_size[0], in_channels * kernel_size[1])
            U, S, V = torch.svd(W)  # 特異値分解
            D = min(rank, min(W.shape))  # ランク
            A = U[:, :D] @ torch.sqrt(torch.diag(S[:D]))
            B = torch.sqrt(torch.diag(S[:D])) @ V[:, :D].T
            ch1 = nn.Conv2d(
                in_channels, D, (1, kernel_size[1]), (1, child.stride[1]), (0, child.padding[1]), bias=False
            )  # 第一層
            ch2 = nn.Conv2d(
                D, child.out_channels, (kernel_size[0], 1), (child.stride[0], 1), (child.padding[0], 0), bias=has_bias
            )  # 第二層
            ch1.weight.data = B.reshape((D, 1, in_channels, kernel_size[1])).permute(
                0, 2, 1, 3
            )  # 定理より得た重みの設定
            ch2.weight.data = A.reshape((out_channels, kernel_size[0], D, 1)).permute(
                0, 2, 1, 3
            )  # 定理より得た重みの設定
            if has_bias:
                ch2.bias.data = child.bias
            setattr(module, name, nn.Sequential(ch1, ch2))  # 畳み込み層を分解した 2 つの層に置き換える
        else:
            low_rank_recursively(child, rank)  # 再帰的に探索

In [4]:
low_rank_recursively(model, rank=32) # ランク 32 の低ランク近似

start = time.time()
print(evaluator.eval(model, torch.device("cpu")))
# CIFAR-10 精度 88.27 %
print(time.time() - start)
# Intel Core i7-12700 で 16.0 秒

0.8827
22.286393880844116


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from eval_cifar10 import Eval

In [6]:
transform_train = transforms.Compose(
    [
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ]
)

batch_size = 256
trainset = torchvision.datasets.CIFAR10(root="./data", train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)

device = "cuda:0" if torch.cuda.is_available() else "cpu"
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=0.01, nesterov=True)

epoch = 10
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epoch)

evaluator = Eval(batch_size=batch_size)

Files already downloaded and verified
Files already downloaded and verified


In [7]:
for epoch in range(epoch):
    model.train()
    for data in trainloader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    lr_scheduler.step()

    acc = evaluator.eval(model, device)

    print(f"Epoch {epoch + 1}, Accuracy: {acc}")

# 10 エポックファインチューニング
# CIFAR-10 精度 92.49 %

Epoch 1, Accuracy: 0.9012
Epoch 2, Accuracy: 0.912
Epoch 3, Accuracy: 0.9101
Epoch 4, Accuracy: 0.9101
Epoch 5, Accuracy: 0.9101
Epoch 6, Accuracy: 0.9168
Epoch 7, Accuracy: 0.9203
Epoch 8, Accuracy: 0.9224
Epoch 9, Accuracy: 0.9248
Epoch 10, Accuracy: 0.9247
